# 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import re
%load_ext autoreload
%autoreload 2

In [2]:
path = r'D:\Repo\WebScraping\JobsDB\jobsdb_data.csv'
df = pd.read_csv(path)

# 2. Data Extraction

## 2.1 Salary

In [3]:
df.loc[df.salary.notna()]

,position,company,location,salary,posted_datetime,education_level,year_of_work,job_highlight,job_desc,qualification,job_level,job_functions,job_type,company_website,company_overview,company_industry,company_benefits_incentives,url
9,Data Analyst (Commercial),"National Healthcare Systems Co., Ltd.",Bangkapi,"THB 25,000 - THB 35,000 /month",NaN,Degree,1.0,"Data Analyst, Business Strategy./Strong in Ana...",{},{'Objective of the positionTo combine technica...,Entry Level,"Information Technology (IT), DBA, MIS, Data Sc...","Full Time, Permanent",http://www.nhealth-asia.com,Join one of the biggest networking healthcare ...,Health & Beauty Care,"Free shuttle bus, Medical insurance, Overtime ...",https://th.jobsdb.com//th/en/job/data-analyst-...
12,Machine Learning Manager,"Charoen Pokphand Group Co., Ltd.",Huaykwang,"THB 70,000 - THB 120,000 /month",NaN,Degree,3.0,"Machine Learning/AI, Python, R, JAVA Programmi...",{},{},Middle,"Information Technology (IT), DBA, MIS, Data Sc...","Full Time, Permanent",http://www.cpgroupglobal.com,"Charoen Pokphand Group Co., Ltd, a leading com...",Manufacturing,"Life insurance, Medical insurance, Performance...",https://th.jobsdb.com//th/en/job/machine-learn...
25,Python Developer (Data Engineer) WFH 100%,"Adastra (Thailand) Co., Ltd.",Pathumwan,"THB 55,000 - THB 120,000 /month",NaN,Degree,2.0,Work from anywhere/Good Python Programming ski...,{'Servant Leadership': ['Technical Requirement...,"{'Job DescriptionAs a\xa0Python Developer,\xa0...",Middle,"Information Technology (IT), Programming / Sof...","Full Time, Permanent",https://bg.adastragrp.com/contacts/,Adastra Thailand is a company within Adastra G...,Management Consultancy/Service,"Dental insurance, Medical insurance, Performan...",https://th.jobsdb.com//th/en/job/python-develo...
37,Automation - Business Intelligence and Web Dev...,"Carenet International Co., Ltd.",Jatuchak,"THB 45,000 - THB 55,000 /month",NaN,Degree,NaN,Proficiency in Python./Adept at the use of BI ...,{},{},Entry Level,"Information Technology (IT), Programming / Sof...",Full Time,NaN,CARENET GROUP is one of a leading recruitment ...,Human Resources Management/Consultancy,NaN,https://th.jobsdb.com//th/en/job/automation-bu...
62,Lead Data Analyst – Digital marketing consult...,Cathcart Associates Asia Recruitment Ltd.,Klongtoey,"THB 70,000 - THB 120,000 /month",NaN,Degree,5.0,Strong team leading/business strategy expertis...,{'Salary': ['Lead a team of developers and bus...,"{""Job summarySalary up to 130,000 THB + Great ...",Senior,"Information Technology (IT), MIS, Marketing / ...","Full Time, Permanent",https://cathcarttechnology.co.th/,Cathcart Technology is an agile and independen...,Human Resources Management/Consultancy,"Dental insurance, Life insurance, Medical insu...",https://th.jobsdb.com//th/en/job/lead-data-ana...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587,Solution Architect (Central Ladprao - Hybrid),Coforge,Ladprao,"Above THB 120,000 /month",NaN,Degree,7.0,5 years+ in Solution Architecture and Design R...,{'Job Descriptions:': ['Bridge the gap between...,{'NIIT Technologies is now\xa0Coforge.Coforge ...,Middle,"Information Technology (IT), Application Speci...","Full Time, Permanent",https://www.coforgetech.com/,NIIT Technologies is now\nCoforge.\nCoforge is...,Information Technology,"Life insurance, Medical insurance, Overtime pa...",https://th.jobsdb.com//th/en/job/solution-arch...
589,Java Developer,Cathcart Associates Asia Recruitment Ltd.,Bangkok,"THB 35,000 - THB 45,000 /month",NaN,Degree,1.0,Java Developer/BTS + Good Bonus/Good environment,"{'Responsibilities:': ['Pimpetch (Pim) /', '09...","{""Location: Bangkok - BTSTHB 35,000 -45,000 /m...",Entry Level,"Information Technology (IT), Application Speci...","Full Time, Permanent",https://cathcarttechnology.co.th/,Cathcart Technology is an agile and independen...,Human Resources Management/Consultancy,"Housing allowance, Life insurance, Medical ins...",https://th.jobsdb

### 2.1.1 Salary range

In [4]:
def extract_salary_range(X):
    if pd.isna(X) or not isinstance(X, str):
        return None

    X = X.replace('\xa0', ' ')
    above_match = re.match(r'Above THB (\d{1,3}(?:,\d{3})?) /month', X)
    if above_match:
        min_salary = int(above_match.group(1).replace(',', ''))
        max_salary = None
    else:
        pattern = r'THB (\d{1,3}(?:,\d{3})?) - THB (\d{1,3}(?:,\d{3})?) /month'
        match = re.search(pattern, X)

        if match:
            min_salary = int(match.group(1).replace(',', ''))
            max_salary = int(match.group(2).replace(',', ''))
        else:
            return None

    avg_salary = None
    if min_salary and max_salary:
        avg_salary = (min_salary + max_salary) / 2
    return min_salary, max_salary, avg_salary

In [5]:
df[['min_salary', 'max_salary','avg_salary']] = df['salary'].apply(lambda x: pd.Series(extract_salary_range(x) if x else [None, None, None]))


In [6]:
df.loc[df.salary.notna()][['min_salary', 'max_salary','avg_salary']]

,min_salary,max_salary,avg_salary
9,25000.0,35000.0,30000.0
12,70000.0,120000.0,95000.0
25,55000.0,120000.0,87500.0
37,45000.0,55000.0,50000.0
62,70000.0,120000.0,95000.0
...,...,...,...
587,120000.0,NaN,NaN
589,35000.0,45000.0,40000.0
590,25000.0,35000.0,30000.0
592,160000.0,NaN,NaN


## Test the Regular Expression Pattern

In [7]:
strings = ['IT Manager (Infrastructure) (100 - 120K) - Near BTS Thong Lor Station',
 'IT Director (Req. Exp. in Infrastructure from Manufacturing) / Pracha U-thit Rd. / 100K-130K',
 'Application Manager - Manufacturing and Supply Chain Base in Thonglor (100 - 120K)',
 'Data Analytics Strategist (Exp data analytics, Data digital transformation/50 - 120K/MRT Rama 9)',
 'Data Analyst  (33K–42K) (SDG-100818)/นักวิเคราะห์ข้อมูล',
 'Business Intelligence & Data Analytics Manager - Supply Chain (65K, Bonus 4 months)',
 'Sr. Business Analyst, Digital and Solutions, 100K (on-site)',
 'CTO - Public Company (350-450K)',
 'Continuous Improvement Manager  - Supply Chain (75K, Bonus 4 months)',
]

In [8]:
all_salaries = []

for string in strings:
    salary_pattern_range = re.compile(r'(\d{1,3}(?:,\d{3})?)-(\d{1,3}(?:,\d{3})?)')
    salary_pattern_single = re.compile(r'(\d{1,3}(?:,\d{3})?)K')
    # Search for the pattern in the position string
    match_range = salary_pattern_range.search(string)
    match_single = salary_pattern_single.search(string)
    
    if match_range:
        min_salary = int(match_range.group(1).replace(',', ''))
        max_salary = int(match_range.group(2).replace(',', ''))
        all_salaries.append({'min_salary': min_salary, 'max_salary': max_salary})
    elif match_single:
        single_salary = int(match_single.group(1).replace(',', ''))
        all_salaries.append({'min_salary': single_salary, 'max_salary': single_salary})


In [9]:
all_salaries

[{'min_salary': 120, 'max_salary': 120},
 {'min_salary': 100, 'max_salary': 100},
 {'min_salary': 120, 'max_salary': 120},
 {'min_salary': 120, 'max_salary': 120},
 {'min_salary': 33, 'max_salary': 33},
 {'min_salary': 65, 'max_salary': 65},
 {'min_salary': 100, 'max_salary': 100},
 {'min_salary': 350, 'max_salary': 450},
 {'min_salary': 75, 'max_salary': 75}]

### 2.1.1 Salary from 'position' column

In [10]:
def extract_salary_range_from_position(X):
    if X['min_salary'] is None or X['max_salary'] is None:
        string = str(X['position'])
        salary_pattern_range = re.compile(r'(\d{1,3}(?:,\d{3})?)-(\d{1,3}(?:,\d{3})?)')
        salary_pattern_single = re.compile(r'(\d{1,3}(?:,\d{3})?)K')
        # Search for the pattern in the position string
        match_range = salary_pattern_range.search(string)
        match_single = salary_pattern_single.search(string)
        
        if match_range:
            min_salary = int(match_range.group(1).replace(',', '')) *1000
            max_salary = int(match_range.group(2).replace(',', '')) *1000
            
            return pd.Series({'min_salary': min_salary, 'max_salary': max_salary})
        elif match_single:
            single_salary = int(match_single.group(1).replace(',', ''))*1000

            return pd.Series({'min_salary': single_salary, 'max_salary': single_salary})
       
        else:
            return pd.Series({'min_salary': None, 'max_salary': None})
    else:
        return pd.Series({'min_salary': X['min_salary'], 'max_salary': X['max_salary']})


In [11]:
df[['min_salary', 'max_salary']] = df.apply(lambda x: extract_salary_range_from_position(x), axis=1)

In [12]:
df.loc[df.min_salary.notna()][['min_salary', 'max_salary','avg_salary']]

,min_salary,max_salary,avg_salary
9,25000.0,35000.0,30000.0
12,70000.0,120000.0,95000.0
25,55000.0,120000.0,87500.0
37,45000.0,55000.0,50000.0
62,70000.0,120000.0,95000.0
...,...,...,...
587,120000.0,NaN,NaN
589,35000.0,45000.0,40000.0
590,25000.0,35000.0,30000.0
592,160000.0,NaN,NaN


## 2.2 Seniority 
| Job Title | Work Experience in Years |
|-----------|--------------------------|
| Junior    | 0-2                      |
| Medior    | 2-5                      |
| Senior    | 5+                       |

Reference: [Link](https://magnet.me/guide/en/the-difference-between-junior-medior-and-senior/#:~:text=What's%20the%20difference%20between%20a,depth%20knowledge%20of%20their%20field.)

In [13]:
def extract_seniority(X):
    position_lower = X['position'].lower()

    # Define regular expressions for seniority levels
    intern_pattern = re.compile(r'\b(intern|internship|นักศึกษาฝึกงาน)\b')
    junior_pattern = re.compile(r'\b(junior|jr.|entry|new grad|เด็กจบใหม่|จบใหม่)\b')
    medior_pattern = re.compile(r'\b(medior|intermediate|มีประสบการณ์|experienced)\b')
    senior_pattern = re.compile(r'\b(senior|sr.|lead|leader|head|header|manager|ผู้จัดการ|อาวุโส|expert)\b')

    # Check for matches using regular expressions and work experience
    if intern_pattern.search(position_lower):
        return 'Intern'
    elif junior_pattern.search(position_lower) or (X['year_of_work'] >= 0 and X['year_of_work'] <= 2):
        return 'Junior'
    elif medior_pattern.search(position_lower) or (X['year_of_work'] > 2 and X['year_of_work'] <= 5):
        return 'Medior'
    elif senior_pattern.search(position_lower) or (X['year_of_work'] > 5):
        return 'Senior'
    else:
        return 'Not specified'

In [14]:
df['seniority'] = df.apply(lambda x: extract_seniority(x), axis=1)

In [15]:
df['seniority'].value_counts()

seniority
Medior           277
Junior           126
Not specified    108
Senior           108
Intern             2
Name: count, dtype: int64

## 2.3 Work location
- Hybrid
- Work from home
- on-site

In [16]:
df

,position,company,location,salary,posted_datetime,education_level,year_of_work,job_highlight,job_desc,qualification,...,job_type,company_website,company_overview,company_industry,company_benefits_incentives,url,min_salary,max_salary,avg_salary,seniority
0,Data Scientist,GMM Grammy Public Company Limited,Wattana,NaN,2023-12-13,Degree,1.0,1-3 years of experience in Data Scientist/Adva...,{'Key Responsibilities:': ['Perform large-scal...,"{'Briefly, we are looking for passionate data ...",...,Permanent,http://www.gmmgrammy.com,NaN,Entertainment/Recreation,"Dental insurance, Life insurance, Medical insu...",https://th.jobsdb.com//th/en/job/data-scientis...,NaN,NaN,NaN,Junior
1,Data Scientist,KASIKORNBANK PUBLIC COMPANY LIMITED,Bangkok,NaN,2023-12-10 00:00:00,Degree,5.0,Data Scientist / Analytics Modeling/Experience...,{'Qualification:': ['Develop fraud management ...,{'Job Title : Data Scientist (Fraud Management...,...,"Full Time, Permanent",NaN,Your Possibilities Begin at KBank \n\n\nCustom...,Financial Services,"Dental insurance, Medical insurance, Performan...",https://th.jobsdb.com//th/en/job/data-scientis...,NaN,NaN,NaN,Medior
2,Officer or Engineer – Hydrology Engineering (...,CK Power Pubblic Comapany Limited,Dindaeng,NaN,2023-12-10 00:00:00,NaN,NaN,Hydrology Data Analyst/Hydropower/Geospatial,{},"{""Summary of Responsibilities:\xa0 \xa0 \xa0 \...",...,Full Time,http://www.xayaburi.com,CKpower จดทะเบียนจัดตั้งเมื่อวันที่ 8 มิถุนายน...,Energy/Power/Water/Oil & Gas/Waste Management,NaN,https://th.jobsdb.com//th/en/job/officer-or-en...,NaN,NaN,NaN,Not specified
3,Perfect Store Manager (Data Scientist),PepsiCo,Klongtoey,NaN,2023-12-11 00:00:00,Master,5.0,"Masters degree in Business, Marketing,Engineer...",{'Main purpose of the role': ['\xa0\xa0\xa0\xa...,{'Main purpose of the roleThis individual will...,...,"Full Time, Permanent",http://www.PepsiCo.com,About PepsiCo\nPepsiCo products are enjoyed by...,Trading and Distribution,"Dental insurance, Life insurance, Five-day wor...",https://th.jobsdb.com//th/en/job/perfect-store...,NaN,NaN,NaN,Medior
4,"Data Scientist, Credit Risk Model (True Money)","Ascend Group Co., Ltd.",Bangkok,NaN,2023-12-10 00:00:00,Degree,2.0,Good Location/Attractive Benefit/Friendly Envi...,{'Responsibilities\xa0': ['Develop and maintai...,{'Job DescriptionWe are looking for a data sci...,...,"Full Time, Permanent",NaN,\nAbout Ascend Group\nAscend Group is more tha...,Information Technology,"Dental insurance, Life insurance, Medical insu...",https://th.jobsdb.com//th/en/job/data-scientis...,NaN,NaN,NaN,Junior
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,Cybersecurity Analyst,"GPV Asia (Thailand) Co., Ltd",Samutprakarn > Muang,NaN,2023-11-28 00:00:00,Degree,1.0,Experience in security Operation Center/Backgr...,{},{'Prime Function :2nd level support of company...,...,"Full Time, Permanent",http://www.gpv.dk,"GPV ASIA (THAILAND) CO., LTD.\nGPV Asia (Thail...",Electronics/Electrical Equipment,"Free shuttle bus, Life insurance, Medical insu...",https://th.jobsdb.com//th/en/job/cybersecurity...,NaN,NaN,NaN,Junior
617,Associate Client Relations Specialist - FSG Bu...,SS&C FinTech Services (Thailand) Limited,Wattana,NaN,2023-11-28 00:00:00,Degree,1.0,Experience in application support role/Have kn...,{},"{""Provides daily client support, including res...",...,"Full Time, Permanent",http://www.ssctech.com,"SS&C Technologies, Inc. (SS&C)\nwas founded in...",Information Technology,"Dental insurance, Life insurance, Medical insu...",https://th.jobsdb.com//th/en/job/associate-cli...,NaN,NaN,NaN,Junior
618,New Grad - Java Software Engineer / Programmer...,Krungsri Bank (Bank of Ayudhya PCL.),Yannawa,NaN,2023-11-28 00:00:00,Degree,NaN,"Software Engineer , Programmer , Developer/Jav...",{'Responsibilities\xa0': ['New grad of strong ...,{'Software Engineer works as a technical devel...,...,"Full Time, Permanent",http://www.krungsri.com,"Krungsri is a strategic member of MUFG Bank

In [17]:
def extract_work_location(X, columns):
    position_lower = ' '.join(str(X[col]) for col in ['position'] + list(columns)).lower()

    # Define regular expressions for work locations
    hybrid_pattern = re.compile(r'\b(hybrid)\b')
    wfh_pattern = re.compile(r'\b(work from home|wfh)\b')
    onsite_pattern = re.compile(r'\b(on-site|onsite)\b')

    # Check for matches using regular expressions
    if hybrid_pattern.search(position_lower):
        return 'Hybrid'
    elif wfh_pattern.search(position_lower):
        return 'Work from home'
    elif onsite_pattern.search(position_lower):
        return 'On-site'
    else:
        return 'Not specified'

In [18]:
df['work_location'] = df.apply(lambda x: extract_work_location(X=x, columns=['position', 
                                                                             'job_highlight', 
                                                                             'company_benefits_incentives']), axis=1)

In [19]:
df['work_location'].value_counts()

work_location
Not specified     375
Work from home    162
Hybrid             77
On-site             7
Name: count, dtype: int64

## 2.3 DA, DE, DS 

In [20]:
def extract_data_ist(X, columns):
    position_lower = ' '.join(str(X[col]) for col in ['position'] + list(columns)).lower()

    # Define regular expressions for work locations
    da = re.compile(r'\b(data engineer)\b')
    de = re.compile(r'\b(data analysis|data analyst|นักวิเคราะห์ข้อมูล|วิเคราะห์ข้อมูล|data analytic(s)?)\b')
    ds = re.compile(r'\b(data science|data scientist|นักวิทยาศาสตร์ข้อมูล|ai|ml|modeler)\b')
    dsp = re.compile(r'\b(data specialist)\b')
    dq = re.compile(r'\b(data quality)\b')

    # Check for matches using regular expressions
    if da.search(position_lower):
        return 'Data Analyst'
    elif de.search(position_lower):
        return 'Data Engineer'
    elif ds.search(position_lower):
        return 'Data Scientist'
    elif dsp.search(position_lower):
        return 'Data Specialist'
    elif dq.search(position_lower):
        return 'Data Quality'
    else:
        return 'Other'

In [21]:
df['data_ist'] = df.apply(lambda x: extract_data_ist(X=x, columns=['position']), axis=1)

In [22]:
df['data_ist'].value_counts()

data_ist
Other             520
Data Engineer      46
Data Analyst       30
Data Scientist     23
Data Quality        2
Name: count, dtype: int64

## 2.4 Extract Date, Month, Year

In [23]:
df['posted_datetime'] = pd.to_datetime(df['posted_datetime'], errors='coerce')

df['hour']=pd.to_datetime(df['posted_datetime']).dt.hour
df['day'] = pd.to_datetime(df['posted_datetime']).dt.dayofweek
df['month']=pd.to_datetime(df['posted_datetime']).dt.month

In [33]:
df.loc[df['hour'].notna() & df['day'].notna()  & df['month'].notna() ][['hour', 'day', 'month']]


,hour,day,month
0,0.0,2.0,12.0
7,0.0,2.0,12.0
8,0.0,2.0,12.0
10,0.0,1.0,12.0
11,0.0,2.0,12.0
...,...,...,...
464,0.0,2.0,12.0
466,0.0,2.0,12.0
484,0.0,2.0,12.0
497,0.0,2.0,12.0


# Export

In [24]:
df.to_csv('jobsdb_data_prep.csv', index=False)